# PySharpe Quickstart

This notebook runs entirely offline using the seeded fixtures included with the repository. It demonstrates how to:

1. Import PySharpe and supporting libraries.
2. Load a small portfolio definition (no network access required).
3. Compute returns, volatility, and Sharpe ratios with the metrics helpers.
4. Optimise a portfolio (when PyPortfolioOpt is available).
5. Plot a simple performance chart.


In [ ]:
from __future__ import annotations

from pathlib import Path
from tempfile import TemporaryDirectory

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pysharpe
from pysharpe import metrics


In [ ]:
# Load tickers from the sample CSV shipped with the tests directory.
portfolio_path = Path('tests/data/sample_portfolio.csv')
raw_tickers = portfolio_path.read_text(encoding='utf-8').splitlines()
portfolio_tickers = [ticker.strip() for ticker in raw_tickers if ticker and not ticker.startswith('#')]
print(f'Loaded tickers: {portfolio_tickers}')

# Build a deterministic price frame using a seeded random walk.
rng = np.random.default_rng(seed=41)
dates = pd.date_range('2022-01-01', periods=90, freq='B')
shocks = rng.normal(loc=0.001, scale=0.02, size=(len(dates), len(portfolio_tickers)))
prices = 100 * np.exp(np.cumsum(shocks, axis=0))
price_frame = pd.DataFrame(prices, index=dates, columns=portfolio_tickers)
price_frame.head()


In [ ]:
# Compute daily returns and annualised statistics.
returns = metrics.compute_returns(price_frame)
annual_return = metrics.annualize_return(returns)
annual_volatility = metrics.annualize_volatility(returns)
sharpe = metrics.sharpe_ratio(returns)

summary = pd.DataFrame({
    'annual_return': annual_return,
    'annual_volatility': annual_volatility,
    'sharpe_ratio': sharpe,
}).sort_index()
summary


In [ ]:
# Run an optimisation using the generated data when PyPortfolioOpt is installed.
try:
    import pypfopt  # noqa: F401
except ModuleNotFoundError:
    print('PyPortfolioOpt is not installed in this environment; skipping optimisation demo.')
else:
    with TemporaryDirectory() as tmpdir:
        tmpdir_path = Path(tmpdir)
        collated = price_frame.copy()
        collated.index.name = 'Date'
        collated.reset_index().to_csv(tmpdir_path / 'demo_collated.csv', index=False)

        result = pysharpe.optimise_portfolio(
            'demo',
            collated_dir=tmpdir_path,
            output_dir=tmpdir_path,
            make_plot=False,
        )
        print(result.summary)


In [ ]:
# Plot cumulative performance to visualise the synthetic series.
figure, ax = plt.subplots(figsize=(8, 4))
(price_frame / price_frame.iloc[0]).plot(ax=ax)
ax.set_title('Synthetic Portfolio Prices (Normalised)')
ax.set_ylabel('Growth (x starting value)')
ax.grid(True)
plt.tight_layout()
figure
